# Projekt Zaliczeniowy z Hurtowni Danych

### Autorzy: Weronika Macholl i Maciej Ossowski

In [1]:
# Importowanie bibliotek
import pyodbc
import requests
from bs4 import BeautifulSoup
import json

# Ustawienia dla lokalnego komputera i bazy danych
local_server   = #enter your local server 
local_database = 'Projekt_Hurtownie_Danych'
local_username = 'Maciej'
local_password = 'Hurtowniedanych'

# Lista słów/fras, które chcesz wyszukać w Google
search_terms = ['python', 'machine learning', 'data science', 'pizza', 'dogs', 'cats', 'football']

In [2]:
class DatabaseHandler:
    """
    Klasa DatabaseHandler obsługuje połączenie oraz operacje na bazie danych SQL Server.

    Argumenty konstruktora:
    server (str): Adres serwera bazy danych.
    database (str): Nazwa bazy danych.
    username (str): Nazwa użytkownika.
    password (str): Hasło użytkownika.
    driver (str): Nazwa sterownika bazy danych (domyślnie '{ODBC Driver 17 for SQL Server}').

    Metody:
    - connect(): Nawiązuje połączenie z bazą danych.
    - execute_query(query: str): Wykonuje zapytanie do bazy danych.
    - fetch_data(query: str): Pobiera dane z bazy danych na podstawie zapytania.
    - close_connection(): Zamyka połączenie z bazą danych.

    Atrybuty:
    - server (str): Adres serwera bazy danych.
    - database (str): Nazwa bazy danych.
    - username (str): Nazwa użytkownika.
    - password (str): Hasło użytkownika.
    - driver (str): Nazwa sterownika bazy danych.
    - connection: Obiekt reprezentujący połączenie z bazą danych.
    - cursor: Kursor do wykonywania poleceń w bazie danych.

    Metoda connect() otwiera połączenie z bazą danych na podstawie dostarczonych danych logowania.
    Metoda execute_query(query) wykonuje podane zapytanie SQL na połączonej bazie danych.
    Metoda fetch_data(query) pobiera dane z bazy danych na podstawie podanego zapytania.
    Metoda close_connection() zamyka otwarte połączenie z bazą danych.

    Przykład użycia:
    db_handler = DatabaseHandler('server_name', 'database_name', 'username', 'password')
    db_handler.connect()
    db_handler.execute_query('SELECT * FROM table_name')
    data = db_handler.fetch_data('SELECT column1, column2 FROM table_name')
    db_handler.close_connection()
    """
    
    def __init__(self, server, database, username, password, driver='{ODBC Driver 17 for SQL Server}'):
        # Kod inicjalizujący atrybuty obiektu DatabaseHandler
        self.server = server
        self.database = database
        self.username = username
        self.password = password
        self.driver = driver
        self.connection = None
        self.cursor = None

    def connect(self):
        # Kod łączący z bazą danych
        connection_string = ('DRIVER=' + self.driver + ';SERVER=' + self.server +
                             ';PORT=1433;DATABASE=' + self.database +
                             ';UID=' + self.username + ';PWD=' + self.password + ';Encrypt=no')
        self.connection = pyodbc.connect(connection_string)
        self.cursor = self.connection.cursor()

    def execute_query(self, query):
        # Kod wykonujący zapytanie SQL
        self.cursor.execute(query)
        self.connection.commit()

    def fetch_data(self, query):
        # Kod pobierający dane z bazy danych
        return self.cursor.execute(query).fetchall()

    def close_connection(self):
        # Kod zamykający połączenie z bazą danych
        self.cursor.close()
        self.connection.close()

In [3]:
def google_search(query, local_database_handler):
    """
    Funkcja google_search wykonuje zapytanie w Google, pobiera wyniki wyszukiwania dla określonego zapytania
    i zbiera informacje na temat znalezionych stron.

    Argumenty:
    query (list): Lista zawierająca frazy do wyszukania w Google.
    local_database_handler: Obiekt obsługujący bazę danych, który będzie wykorzystywany do zapisywania wyników.

    Zwraca:
    list: Lista słowników reprezentujących znalezione wyniki wyszukiwania w Google, zawierających:
        - 'title': Tytuł znalezionej strony.
        - 'url': Adres URL znalezionej strony.
        - 'description': Krótki opis strony.
        - 'publication_date': Data publikacji (jeśli dostępna).
        - 'rating': Ocena (jeśli dostępna).
        - 'comments': Komentarze (jeśli dostępne).
        - 'similar_urls': Lista podobnych adresów URL (jeśli dostępne).

    Wyjątki:
    - Błąd pobierania strony Google: Wyświetla komunikat o błędzie i zwraca None.
    - Wystąpił błąd podczas wykonania zapytania: Wyświetla komunikat o błędzie i zwraca None.
    """
    
    url = f"https://www.google.com/search?q={'+'.join(query)}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    }
    results = []
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            search_results = soup.find_all('div', class_='tF2Cxc')
            for result in search_results:
                title = result.find('h3', class_='LC20lb').text
                url = result.find('a')['href']
                description = result.find('div', class_='lyLwlc').text.split('.')[0] if result.find('div', class_='lyLwlc') else ''
                #description = result.find('div', class_='IsZvec').text.split('.')[0] if result.find('div', class_='IsZvec') else ''
                # description = result.find('div', class_='IsZvec').text if result.find('div', class_='IsZvec') else ''
                publication_date = result.find('span', class_='f').text if result.find('span', class_='f') else ''
                rating = result.find('span', class_='GpQGbf').text if result.find('span', class_='GpQGbf') else ''
                comments = result.find('span', class_='slp').text if result.find('span', class_='slp') else ''
                similar_urls = [a['href'] for a in result.find_all('a', class_='fl') if a.get('href')]

                results.append({
                    'title': title,
                    'url': url,
                    'description': description,
                    'publication_date': publication_date,
                    'rating': rating,
                    'comments': comments,
                    'similar_urls': similar_urls
                })
            return results
        else:
            print("Błąd pobierania strony Google.")
            return None
    except requests.RequestException as e:
        print(f"Wystąpił błąd: {e}")
        return None

In [4]:
# Użycie lokalnego środowiska
local_database_handler = DatabaseHandler(
    server   = local_server,
    database = local_database,
    username = local_username,
    password = local_password
)

local_database_handler.connect()

In [5]:
# Usuwanie istniejącej tabeli
drop_table_if_exists_query = '''DROP TABLE IF EXISTS GoogleSearchResults;'''
local_database_handler.execute_query(drop_table_if_exists_query)

# Tworzenie nowej tabeli w bazie danych
create_table_query = '''CREATE TABLE GoogleSearchResults (
                            ID INT IDENTITY(1,1) PRIMARY KEY,
                            SearchTerm VARCHAR(100),
                            Title VARCHAR(255),
                            URL VARCHAR(255),
                            Description VARCHAR(255),
                            PublicationDate VARCHAR(100),
                            Rating VARCHAR(50),
                            Comments VARCHAR(50),
                            SimilarURLs VARCHAR(255)
                        );'''
local_database_handler.execute_query(create_table_query)

In [6]:
# Pobieranie wyników i wstawianie ich do tabeli w bazie danych
for term in search_terms:
    search_results = google_search([term], local_database_handler)
    if search_results:
        for result in search_results:
            # Skracanie długości kolumn do odpowiednich limitów
            title = result['title'][:100]
            url = result['url'][:255]
            description = result['description'][:255]
            publication_date = result['publication_date'][:100]
            rating = result['rating'][:50]
            comments = result['comments'][:50]
            similar_urls = result['similar_urls'][:255]

            insert_query = '''INSERT INTO GoogleSearchResults (SearchTerm, Title, URL, Description, PublicationDate, Rating, Comments, SimilarURLs)
                   VALUES (?, ?, ?, ?, ?, ?, ?, ?)'''
            data_to_insert = (term, title, url, description, publication_date, rating, comments, ", ".join(similar_urls) if similar_urls else '')
            local_database_handler.cursor.execute(insert_query, data_to_insert)
            local_database_handler.connection.commit()

In [7]:
# Pobieranie danych z tabeli w bazie danych
select_query = "SELECT * FROM GoogleSearchResults"
results_from_db = local_database_handler.fetch_data(select_query)

In [8]:
# Przygotowanie listy wyników
results_list = [{'ID': row[0], 'SearchTerm': row[1], 'Title': row[2], 'URL': row[3], 'Description': row[4],
                 'PublicationDate': row[5], 'Rating': row[6], 'Comments': row[7], 'SimilarURLs': row[8]} for row in results_from_db]

# Zapis danych do pliku JSON
with open('google_search_results_from_db.json', 'w', encoding='utf-8') as f:
    json.dump(results_list, f, ensure_ascii=False, indent=4)

In [9]:
# Zamykanie połączenia z bazą danych
local_database_handler.close_connection()

print("Wyniki wyszukiwania zapisano do pliku JSON i wstawiono do tabeli w bazie danych.")

Wyniki wyszukiwania zapisano do pliku JSON i wstawiono do tabeli w bazie danych.
